# Silent Cities : Exemple de traitement d'un fichier audio

In [ ]:
import os
import sys
sys.path.insert(0, "../")

import pandas as pd
import numpy as np
import torch
import torchaudio
from matplotlib import pyplot as plt
import IPython
import librosa
from plotly import express as px

from utils import librosa_display as ld
from utils import utils

from audioset_tagging_cnn.inference import audio_tagging

## Chargement de la base de donnée

In [ ]:
base_de_donnee = pd.read_csv('public_final_metadata_rest.csv')
base_de_donnee

In [ ]:
site =  1 # Choix du site pour la suite
fichier_audio = '5E730358.WAV' # Nom d'un fichier audio

## Extraction des méta données 

In [ ]:
_, meta = utils.read_audio_hdr(fichier_audio, False)
print(f'''Méta donnée d'un fichier audio : {meta}''')

## Chargement d'un fichier audio

In [ ]:
audio, sr = torchaudio.load(fichier_audio)

In [ ]:
print(f'Un fichier audio est composé de points en fonction du temps : {audio}')
print(f'''et d'une fréquence d'echantillonnage {sr} Hz soit {sr} points par seconde.''')

In [ ]:
audio_10sec = audio[0, :int(10*sr)] # Prenons les 10 premières secondes

# Écoute et visualisation d'un fichier audio

In [ ]:
IPython.display.Audio(fichier_audio)

In [ ]:
plt.figure()
plt.plot(torch.arange(int(10*sr))/sr, audio_10sec)
plt.xlabel('temps (s)'); plt.ylabel('amplitude (-)'); plt.title('Représentation temporelle')
plt.show()

In [ ]:
# mel-spectrogramme
Zxx = librosa.feature.melspectrogram(y=audio_10sec.numpy(), sr=sr, n_mels=256, fmax=20000)
t, f, Zxx = ld.specshow(Zxx, sr = sr, fmax = 20000,y_axis='mel', x_axis='time', ref = np.max)

In [ ]:
plt.figure(figsize = (10,4))
plt.subplot(121)
plt.pcolormesh(t, f, 10*np.log10(Zxx))
plt.yscale('log'); plt.xlabel('temps (s)'); plt.ylabel('fréquence (Hz)')
plt.colorbar(label='Énergie normalisé (dB)'); plt.title('Mel-spectrogramme')
plt.ylim(100, 10000)

plt.subplot(122)
plt.pcolormesh(t, f, 10*np.log10(Zxx))
plt.xlabel('temps (s)'); plt.ylabel('fréquence (Hz)'); plt.title('Mel-spectrogramme')
plt.colorbar(label='Énergie (dB)')
plt.ylim(100, 10000)
plt.tight_layout()
plt.show()
# Vous préférez la quelle ?

## Passage dans le réseau

In [ ]:
checkpoint_path = 'ResNet22_mAP=0.430.pth'
with torch.no_grad():
    proba, labels, sorted_indexes, _ = audio_tagging(audio_10sec[None,:], checkpoint_path , usecuda=False)

In [ ]:
sortie_reseau = pd.DataFrame(columns = ['label', 'proba (%)'])
sortie_reseau['label'] = labels
sortie_reseau['proba (%)'] = proba[0,:]*100
sortie_reseau

## Visualisation de la sortie du réseau et des méta données associées

In [ ]:
fig = px.scatter(sortie_reseau, x = 'label', y='proba (%)',# color='proba (%)',
                 title = f'''Fichier : {fichier_audio},<br>
                             Date et heure : {meta['date']} {meta['time']},<br>
                             Pays : {base_de_donnee['country'][site]},
                             Ville : {base_de_donnee['city'][site]},
                             Numéro du site : {site}''')
fig.show()